In [1]:
import sys
import os
# sys.path.append("C:\\Program Files\\ASAP 2.1\\bin")
sys.path.append("D:\\AB\\DigiPath-Tools\\Docker_Code\\app\\annotation2overlay")

vipshome = 'C:\\vips-dev-8.15\\bin'  ## Path to your VIPS library
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
# import multiresolutionimageinterface as mir
import numpy as np
import pyvips
# import openslide
# from PIL import Image
import cv2 as cv
from xml2json import xml2json
from json2binary import json2binary

In [2]:
relevant_path = os.getcwd()
image_extension = ['tif']
Anno_extension = ['xml']
Json_extension = ['json']

Anno_Source_Folder = "E:\\Camelyon17\\annotations"##"D:\\AB\\ASAP_Annotation_Mask\\Json_Annotations"
WSI_Source_Folder = "E:\\Camelyon17\\images"  #os.path.join(relevant_path, "Delineated")

#WSIs = [fn for fn in os.listdir(Source_Folder) if any(fn.endswith(ext) for ext in image_extension)]

Annotations = [fn for fn in os.listdir(Anno_Source_Folder) if any(fn.endswith(ext) for ext in Anno_extension)]

Mask_output = os.path.join(relevant_path, "Exclusion")
if os.path.exists(Mask_output):
    print("Mask output directory already exists.")
else:
    os.mkdir(Mask_output)

In [3]:
# for F in Annotations:
#     xml_path = os.path.join(Anno_Source_Folder, F)
    
    
#     Json = xml2json(xml_path, output_json=True)
#     break

In [4]:
for F in Annotations:
    F = F[:-4]
    print(F)
    
    if os.path.isfile(os.path.join(WSI_Source_Folder, F+".tif")):
        
        output_mask = os.path.join(Mask_output, F+"-Mask.tif")
        if os.path.isfile(output_mask[:-3]+"tif"):
            print("Mask Already Exists")
        else:
        
            image = pyvips.Image.new_from_file(os.path.join(WSI_Source_Folder, F+".tif"))
    
            dimensions = (image.height, image.width) ##(rows, cols)
            print(dimensions)
            xml_path = os.path.join(Anno_Source_Folder, F+".xml")
            Json = xml2json(xml_path, output_json=False)
    
            annos = []
    
            for k in Json.keys():
                annos.append(Json[k])
    
            bin_mask = json2binary(annos, dimensions, Group = 'exclusion')
    
            img = pyvips.Image.new_from_array(bin_mask)
            img.tiffsave(
                        output_mask,
                        pyramid=True,
                        tile=True,
                        bigtiff=True,
                        subifd=False,
                        compression="jpeg",
                        Q=75,
                        tile_height=1024,
                        tile_width=1024,
                        xres=image.xres,
                        yres=image.yres)
#     break

patient_004_node_4
(197226, 96651)
patient_009_node_1
(210579, 94968)
patient_010_node_4
(197226, 96651)
patient_012_node_0
(197226, 96651)
patient_015_node_1
(210579, 94968)
patient_015_node_2
(197226, 96651)
patient_016_node_1
(197226, 96651)
patient_017_node_1
(197226, 96651)
patient_017_node_2
(210579, 94968)
patient_017_node_4
(197226, 96651)
patient_020_node_2
(210579, 94968)
patient_020_node_4
(210579, 94968)
patient_021_node_3
(210579, 94968)
patient_022_node_4
(210579, 94968)
patient_024_node_1
(210579, 94968)
patient_024_node_2
(210579, 94968)
patient_034_node_3
(210579, 94968)
patient_036_node_3
(210579, 94968)
patient_038_node_2
(210579, 94968)
patient_039_node_1
(210579, 94968)
patient_040_node_2
(103936, 176128)
patient_041_node_0
(103936, 147456)
patient_042_node_3
(100352, 139264)
patient_044_node_4
(86016, 57344)
patient_045_node_1
(100352, 118784)
patient_046_node_3
(93184, 131072)
patient_046_node_4
(89600, 65536)
patient_048_node_1
(103936, 143360)
patient_051_node_

In [5]:

for F in Annotations:
    F = F[:-4]
    reader = mir.MultiResolutionImageReader()
    
    WSI_Path = os.path.join(WSI_Source_Folder, F+".tif")
    
    mr_image = reader.open(WSI_Path)
    
    annotation_list = mir.AnnotationList()
    xml_repository = mir.XmlRepository(annotation_list)
    xml_repository.setSource(F+".xml")
    xml_repository.load()
    
    annotation_mask = mir.AnnotationToMask()

    output_path = os.path.join(Mask_output, F+"-Mask.tif")
    A = annotation_mask.convert(annotation_list, output_path, mr_image.getDimensions(), mr_image.getSpacing())
    #print(type(A))
    
    # W = openslide.OpenSlide(output_path)
    # Img = W.read_region((0,0), 5, W.level_dimensions[5])
    # Img.save(output_path[:-3]+"png")
    # W.close()
    # os.remove(output_path)

NameError: name 'mir' is not defined

In [ ]:
Wsi_Lvl = 5

for F in WSIs:
    file_name = os.path.join(Source_Folder, F)
    print(file_name)
    W = openslide.OpenSlide(file_name)
    #Img, Seg = Otsu_Seg(W, Wsi_Lvl, Otsu_output)
    Img = cv.imread(os.path.join(Otsu_output, F[:-4]+".jpg"))
    Seg = cv.imread(os.path.join(Otsu_output, F[:-4]+"-Mask.png"), 0)
    
    Anno_Path = os.path.join(Mask_output, F[:-4]+"-Mask.png")
    Anno_Mask = cv.imread(Anno_Path, 0)
    #Anno_Mask = Anno_Mask / 1.0    
    Final_Mask = cv.bitwise_and(Seg, Anno_Mask)
    cv.imwrite(os.path.join(FInal_Mask_output, F[:-4] + "-FinalMask.png"), (Final_Mask*255))
    
    W.close()
    del Img, Seg, Anno_Mask, Final_Mask, file_name, Anno_Path